Esta notebook contiene bloques de código útiles para realizar Q-learning en el entorno "Pendulum"

In [37]:
import numpy as np
from pendulum_env_extended import PendulumEnvExtended
import random 

In [38]:
env = PendulumEnvExtended()

In [35]:
max_steps= env.max_steps
max_steps

700

Discretización de los estados

In [39]:
x_space_partitions = 11
y_space_partitions = 11
vel_space_partitions = 33

In [40]:
x_space = np.linspace(-1, 1, x_space_partitions)
x_space

array([-1. , -0.8, -0.6, -0.4, -0.2,  0. ,  0.2,  0.4,  0.6,  0.8,  1. ])

In [41]:
y_space = np.linspace(-1, 1, y_space_partitions)
y_space

array([-1. , -0.8, -0.6, -0.4, -0.2,  0. ,  0.2,  0.4,  0.6,  0.8,  1. ])

In [42]:

vel_space = np.linspace(-8, 8, vel_space_partitions)
vel_space

array([-8. , -7.5, -7. , -6.5, -6. , -5.5, -5. , -4.5, -4. , -3.5, -3. ,
       -2.5, -2. , -1.5, -1. , -0.5,  0. ,  0.5,  1. ,  1.5,  2. ,  2.5,
        3. ,  3.5,  4. ,  4.5,  5. ,  5.5,  6. ,  6.5,  7. ,  7.5,  8. ])

Obtener el estado a partir de la observación

In [43]:
def get_state(obs):
    x, y, vel = obs
    x_bin = np.digitize(x, x_space)
    y_bin = np.digitize(y, y_space)
    vel_bin = np.digitize(vel, vel_space)
    return x_bin, y_bin, vel_bin

In [44]:
state = get_state(np.array([-0.4, 0.2, 0.3]))
state

(3, 6, 17)

Discretización de las acciones

In [45]:
actions_partitions = 21

In [46]:
actions = list(np.linspace(-2, 2, actions_partitions))
actions

[-2.0,
 -1.8,
 -1.6,
 -1.4,
 -1.2,
 -1.0,
 -0.7999999999999998,
 -0.5999999999999999,
 -0.3999999999999999,
 -0.19999999999999996,
 0.0,
 0.20000000000000018,
 0.40000000000000036,
 0.6000000000000001,
 0.8000000000000003,
 1.0,
 1.2000000000000002,
 1.4000000000000004,
 1.6,
 1.8000000000000003,
 2.0]

In [47]:
def get_sample_action():
    return random.choice(actions)

Inicilización de la tabla Q

In [48]:
Q = np.zeros((len(x_space) + 1, len(y_space) + 1, len(vel_space) + 1, len(actions)))
Q

array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
    

Obtención de la acción a partir de la tabla Q

In [49]:
def optimal_policy(state, Q):
    action = actions[np.argmax(Q[state])]
    return action

Epsilon-Greedy Policy

In [50]:
def epsilon_greedy_policy(state, Q, epsilon=0.1):
    explore = np.random.binomial(1, epsilon)
    if explore:
        action = get_sample_action()
    # exploit
    else:
        action = optimal_policy(state, Q)
        
    return action

Ejemplo de episodio 

In [51]:
obs,_ = env.reset()
print(obs)
done = False
total_reward = 0
state = get_state(obs)
while not done:
    # Acción del modelo
    action = epsilon_greedy_policy(state, Q, 0.5)
    
    # Indice de la accion en Q
    action_idx = actions.index(action)
    
    # Acción del ambiente
    real_action = np.array([action_idx])
     
    obs, reward, done, _, _ = env.step(real_action)
    next_state = get_state(obs)
    
   # Usar action_idx para actualizar Q
   
    total_reward += reward
    print('->', state, action, reward, obs, done)
print('total_reward', total_reward)

[0.83758765 0.5463029  0.662408  ]
exploit
-> (9, 7, 54) -2.0 -0.3818975963952661 [0.8158778  0.57822436 0.7721352 ] False
explore
-> (9, 7, 54) 1.1111111111111107 -0.4409885710009289 [0.77430886 0.6328079  1.3724701 ] False
explore
-> (9, 7, 54) -0.22222222222222232 -0.6578804641425356 [0.7138181  0.70033115 1.8137428 ] False
explore
-> (9, 7, 54) 1.5555555555555554 -0.9333467975132922 [0.61809635 0.78610235 2.5723245 ] False
explore
-> (9, 7, 54) -0.22222222222222232 -1.4798142884160288 [0.4880816 0.872798  3.128568 ] False
explore
-> (9, 7, 54) -1.5555555555555556 -2.106734810242148 [0.3263114 0.9452623 3.549833 ] False
explore
-> (9, 7, 54) -1.5555555555555556 -2.796179752122054 [0.13075098 0.99141526 4.0254464 ] False
exploit
-> (9, 7, 54) -2.0 -3.6970713039343095 [-0.09219286  0.9957412   4.469008  ] False
exploit
-> (9, 7, 54) -2.0 -4.767172219030381 [-0.3317091  0.9433817  4.915814 ] False
exploit
-> (9, 7, 54) -2.0 -6.064463580306398 [-0.5681715  0.8229102  5.32335  ] False
ex

In [49]:
max_steps = env.max_steps

In [52]:
iterations = 10

In [53]:
alpha = 0.01
alpha

0.01

In [54]:
gamma = 0.9
gamma

0.9

In [55]:
episodes=1000

In [56]:
i=0
for i in tqdm(range(episodes)):
    obs,_ = env.reset()
    done = False
    total_reward = 0    
    total_steps = 0 
    state = get_state(obs)
    while not done:        
        action = epsilon_greedy_policy(state, Q, 0.2)
        action_idx = actions.index(action)    
        real_action = np.array([action_idx])

        obs, reward, done, _, _ = env.step(real_action)

        next_state = get_state(obs)

    
        total_steps += 1 
        total_reward += reward

        value = Q[state][action_idx] + alpha * (reward + (gamma* (np.max(Q[next_state]))) - Q[state][action_idx] )
        Q[state][action_idx] = value

        state = next_state
    i+=1

relation -6.927479008251885
total_reward -6927.479008251886
total_steps 1000
relation -7.685851977014525
total_reward -7685.851977014525
total_steps 1000
relation -8.055428364781582
total_reward -8055.428364781582
total_steps 1000
relation -7.6186019141788055
total_reward -7618.601914178806
total_steps 1000
relation -7.950964735317082
total_reward -7950.964735317082
total_steps 1000
relation -8.24085520339379
total_reward -8240.85520339379
total_steps 1000
relation -8.23903580220642
total_reward -8239.03580220642
total_steps 1000
relation -8.243380859666216
total_reward -8243.380859666217
total_steps 1000
relation -8.238779687476374
total_reward -8238.779687476374
total_steps 1000
relation -5.957817512558841
total_reward -5957.817512558841
total_steps 1000
relation -7.0258598499254115
total_reward -7025.859849925411
total_steps 1000
relation -7.4062656306060335
total_reward -7406.2656306060335
total_steps 1000
relation -8.100714530876193
total_reward -8100.714530876192
total_steps 1000

In [ ]:
total_steps

700

In [ ]:
total_reward

-5547.40133382486

In [ ]:
Q

array([[[[  0.        ,   0.        ,   0.        , ...,   0.        ,
            0.        ,   0.        ],
         [  0.        ,   0.        ,   0.        , ...,   0.        ,
            0.        ,   0.        ],
         [  0.        ,   0.        ,   0.        , ...,   0.        ,
            0.        ,   0.        ],
         ...,
         [  0.        ,   0.        ,   0.        , ...,   0.        ,
            0.        ,   0.        ],
         [  0.        ,   0.        ,   0.        , ...,   0.        ,
            0.        ,   0.        ],
         [  0.        ,   0.        ,   0.        , ...,   0.        ,
            0.        ,   0.        ]],

        [[  0.        ,   0.        ,   0.        , ...,   0.        ,
            0.        ,   0.        ],
         [  0.        ,   0.        ,   0.        , ...,   0.        ,
            0.        ,   0.        ],
         [  0.        ,   0.        ,   0.        , ...,   0.        ,
            0.        ,   0.     